In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as sio
from glob import glob
import os
import ordpy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

In [ ]:
def plot_prob_points(*args, facecolors=None, alpha=None, capsize=10, elinewidth=10, capthick=6, dot_size=400, yticks=None, ytickslabels=None, ymin=None, ymax=None, random_state=10, zero_line=False, edge_color="none"):
    np.random.seed(random_state)
    SPREAD_LENGTH = 0.1
    
    x_list = np.arange(1.0, 1.0+0.5*len(args), 0.5)
    y_list = list(map(lambda x: x.mean(), args))
    error_list = list(map(lambda x: x.std()/np.sqrt(len(x)), args))
    spread_list = x_list - SPREAD_LENGTH/2.0
    
    # create points for means and standard errors
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    ax.errorbar(x_list, y_list, error_list, ls="none", capsize=capsize, elinewidth=elinewidth, capthick=capthick, ecolor="black")
    ax.scatter(x_list, y_list, s=dot_size, c="black", zorder=10)
    
    if facecolors is None:
        facecolors = np.repeat("#cccccc", len(args))
    if alpha is None:
        alpha = np.repeat(0.5, len(args))

    if type(facecolors) == str:
        facecolors = np.repeat(facecolors, len(args))
    if type(alpha) == float or type(alpha) == int:
        alpha = np.repeat(alpha, len(args))
    
    if zero_line:
        ax.plot([1.0-0.4, max(x_list)+0.4], [0, 0], c="#cccccc", linestyle="dashed", linewidth=elinewidth, zorder=0)

    # draw each point
    for i, arg in enumerate(args):
        ax.scatter(
            np.random.rand(len(arg))*SPREAD_LENGTH + spread_list[i], 
            arg, 
            c=facecolors[i], 
            edgecolors=edge_color, 
            s=dot_size, 
            alpha=alpha[i],
            zorder=0
        )


    ax.set_xlim([1.0-0.4, max(x_list)+0.4])
    ax.set_xticks(x_list, np.repeat("", len(args)))
    ax.spines[['right', 'top']].set_visible(False)
    plt.setp(ax.spines.values(), linewidth=8)
    
    if ymin and ymax:
        ax.set_ylim([ymin, ymax])
    
    if yticks:
        if ytickslabels:
            ax.set_yticks(yticks, ytickslabels)
        else:
            ax.set_yticks(yticks)

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"
N_rois = 300
subs = sorted(glob(f"{MAIN_DIR}/results/func_networks/ROIs_300inVol_MNI/*"))
N_subs = len(subs)
subs_controles = sorted(glob(f"{MAIN_DIR}/results/func_networks/ROIs_300inVol_MNI_controles/*"))
N_subs_controles = len(subs_controles)

#por red funcional o ROI
network_affiliation = np.loadtxt("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/func_networks/masks/ROIs_300inVol_MNI/ROIs_CommunityAffiliation.txt")
network_names = ["DMN", "VIS", "FP", "Rew", "DA", "VA", "SN", "CO", "SMD", "SML", "AUD", "PM", "MTL", "un"]
network_list = np.unique(network_affiliation)
network_list = network_list[:-1]

he_matrix = np.zeros((N_subs, 4, N_rois)) #subs x conditions
sc_matrix = np.zeros((N_subs, 4, N_rois)) #subs x conditions
he_matrix_network = np.zeros((N_subs, 4, 13)) #subs x conditions
sc_matrix_network = np.zeros((N_subs, 4, 13)) #subs x conditions
he_matrix_controles = np.zeros((N_subs_controles, 4, N_rois)) #subs x conditions
sc_matrix_controles = np.zeros((N_subs_controles, 4, N_rois)) #subs x conditions

data_avp = np.zeros((N_subs, 4, N_rois, 150))
data_data_controles = np.zeros((N_subs_controles, 4, N_rois, 150))
for iSub, sub in enumerate(subs):
    print(f"{os.path.basename(sub)}")
    for iCond in [0,1,2,3]:
        data = sio.loadmat(f"{sub}/cond{iCond+1}/func_roi.mat")["func_roi"]
        for iRoi in range(N_rois):
            if network_affiliation[iRoi] == 18:
                he_matrix[iSub, iCond, iRoi] = np.nan
                sc_matrix[iSub, iCond, iRoi] = np.nan
                continue
                
            he, sc = ordpy.complexity_entropy(data[iRoi,:], dx=3)
            he_matrix[iSub, iCond, iRoi] = he
            sc_matrix[iSub, iCond, iRoi] = sc
            data_avp[iSub, iCond, iRoi, :] = data[iRoi,:]

for iSub, sub in enumerate(subs_controles):
    print(f"{os.path.basename(sub)}")
    for iCond in [0,1,2,3]:
        data_controles = sio.loadmat(f"{sub}/cond{iCond+1}/func_roi.mat")["func_roi"]
        for iRoi in range(N_rois):
            # if network_affiliation[iRoi] == 18:
            #     he_matrix_controles[iSub, iCond, iRoi] = np.nan
            #     sc_matrix_controles[iSub, iCond, iRoi] = np.nan
            #     continue
                
            he, sc = ordpy.complexity_entropy(data_controles[iRoi,:], dx=3)
            he_matrix_controles[iSub, iCond, iRoi] = he
            sc_matrix_controles[iSub, iCond, iRoi] = sc
            data_data_controles[iSub, iCond, iRoi, :] = data_controles[iRoi,:]
            
for iSub, sub in enumerate(subs):
    print(os.path.basename(sub))
    for iCond in [0,1,2,3]:
        data = sio.loadmat(f"{sub}/cond{iCond+1}/func_roi.mat")["func_roi"]
        for iNetwork, Network in enumerate(network_list):
            serie = np.mean(data[network_affiliation == Network, :], axis=0)
                
            he, sc = ordpy.complexity_entropy(serie, dx=3)
            he_matrix_network[iSub, iCond, iNetwork] = he
            sc_matrix_network[iSub, iCond, iNetwork] = sc

REMOVE_REPOSO = 7-1
REMOVE_ALTERACION = 5-1
he_matrix_controles[REMOVE_REPOSO, 0, :] = np.nan
sc_matrix_controles[REMOVE_REPOSO, 0, :] = np.nan
he_matrix_controles[REMOVE_ALTERACION, 2, :] = np.nan
sc_matrix_controles[REMOVE_ALTERACION, 2, :] = np.nan

data_data_controles[REMOVE_REPOSO, 0, :, :] = np.nan
data_data_controles[REMOVE_REPOSO, 0, :, :] = np.nan
data_data_controles[REMOVE_ALTERACION, 2, :, :] = np.nan
data_data_controles[REMOVE_ALTERACION, 2, :, :] = np.nan

he_matrix_controles = he_matrix_controles[:,:,0:288]
sc_matrix_controles = sc_matrix_controles[:,:,0:288]
he_matrix = he_matrix[:,:,0:288]
sc_matrix = sc_matrix[:,:,0:288]

data_data_controles = data_data_controles[:,:,0:288,:]
data_avp = data_avp[:,:,0:288,:]

In [ ]:
def calc_complexity_stats(he_matrix_network, sc_matrix_network):
    
    df_data = {"session": [], "cond": [], "network": [], "he": [], "sc": [], "roi": []}
    for iSub in range(20):
        for iCond in range(4):
            for iRoi in range(288):
                df_data["session"].append(iSub+1)
                df_data["cond"].append(iCond+1)
                df_data["network"].append(network_affiliation[iRoi+12])
                df_data["roi"].append(iRoi)
                df_data["he"].append(he_matrix[iSub, iCond, iRoi+12]) #las primeras 12 son unc.
                df_data["sc"].append(sc_matrix[iSub, iCond, iRoi+12])

    df = pd.DataFrame(df_data)
    df['cond'] = df['cond'].astype('category')
    df['network'] = df['network'].astype('category')
    df['roi'] = df['roi'].astype('category')
    df['session'] = df['session'].astype('category')

    md = smf.mixedlm("he ~ network + cond - 1", df, groups=df["session"])
    mdf = md.fit()
    mdf.summary()

    all_data = pd.DataFrame({"pvalues": mdf.pvalues[:-1], "tvalues": mdf.tvalues[:-1]})
    significant_data = all_data[all_data["pvalues"] < 0.05]

    return all_data, significant_data

all_data_sc, significant_data_sc = calc_complexity_stats(he_matrix_network, sc_matrix_network)
all_data_sc

In [ ]:
h, pcorregido = fdrcorrection(all_data_sc["pvalues"][-3:].values, alpha=0.05, method="indep", is_sorted=False)
h, pcorregido

In [ ]:
def bootstrap_entropy_complexity(condicion: str) -> (float, float):
    condiciones = {"trans": 0, "alter": 1, "recup": 2}
    index = condiciones[condicion]

    he_per_subject = np.nanmean(he_matrix, axis=2)
    sc_per_subject = np.nanmean(sc_matrix, axis=2)
    he_controles_per_subject = np.nanmean(he_matrix_controles, axis=2)
    sc_controles_per_subject = np.nanmean(sc_matrix_controles, axis=2)

    he_diff = np.zeros((20, 3))
    sc_diff = np.zeros((20, 3))
    he_controles_diff = np.zeros((10,3))
    sc_controles_diff = np.zeros((10,3))
    he_diff[:,0] = he_per_subject[:,1] - he_per_subject[:,0]
    he_diff[:,1] = he_per_subject[:,2] - he_per_subject[:,0]
    he_diff[:,2] = he_per_subject[:,3] - he_per_subject[:,0]
    sc_diff[:,0] = sc_per_subject[:,1] - sc_per_subject[:,0]
    sc_diff[:,1] = sc_per_subject[:,2] - sc_per_subject[:,0]
    sc_diff[:,2] = sc_per_subject[:,3] - sc_per_subject[:,0]
    he_controles_diff[:,0] = he_controles_per_subject[:,1] - he_controles_per_subject[:,0]
    he_controles_diff[:,1] = he_controles_per_subject[:,2] - he_controles_per_subject[:,0]
    he_controles_diff[:,2] = he_controles_per_subject[:,3] - he_controles_per_subject[:,0]
    sc_controles_diff[:,0] = sc_controles_per_subject[:,1] - sc_controles_per_subject[:,0]
    sc_controles_diff[:,1] = sc_controles_per_subject[:,2] - sc_controles_per_subject[:,0]
    sc_controles_diff[:,2] = sc_controles_per_subject[:,3] - sc_controles_per_subject[:,0]

    N_perm = 50_000
    d_list_he = np.zeros(N_perm)
    d_list_sc = np.zeros(N_perm)
    for perm in tqdm(range(N_perm)):
        data_he = np.hstack((he_controles_diff[:,index], he_controles_diff[:,index], he_diff[:,index]))
        data_sc = np.hstack((sc_controles_diff[:,index], sc_controles_diff[:,index], sc_diff[:,index]))
        
        np.random.shuffle(data_he)
        np.random.shuffle(data_sc)

        d_he = np.nanmean(data_he[0:20]) - np.nanmean(data_he[20:40])
        d_sc = np.nanmean(data_sc[0:20]) - np.nanmean(data_sc[20:40])

        d_list_he[perm] = d_he
        d_list_sc[perm] = d_sc

    x_he = np.nanmean(he_controles_diff[:,index]) - np.nanmean(he_diff[:,index])
    x_sc = np.nanmean(sc_controles_diff[:,index]) - np.nanmean(sc_diff[:,index])
    z_he = np.sign(x_he)
    z_sc = np.sign(x_sc)

    p_he = np.sum(d_list_he > np.abs(x_he))/len(d_list_he)
    p_sc = np.sum(d_list_sc > np.abs(x_sc))/len(d_list_sc)
    
    return p_he, p_sc, d_list_he, d_list_sc, x_he, x_sc, z_he, z_sc

In [ ]:
p_he = [0, 0, 0]
p_sc = [0, 0, 0]
z_he = [0, 0, 0]
z_sc = [0, 0, 0]
p_he[0], p_sc[0], d_list_he_trans, d_list_sc_trans, x_he_trans, x_sc_trans, z_he[0], z_sc[0] = bootstrap_entropy_complexity("trans")
p_he[1], p_sc[1], d_list_he_alter, d_list_sc_alter, x_he_alter, x_sc_alter, z_he[1], z_sc[1]  = bootstrap_entropy_complexity("alter")
p_he[2], p_sc[2], d_list_he_recup, d_list_sc_recup, x_he_recup, x_sc_recup, z_he[2], z_sc[2] = bootstrap_entropy_complexity("recup")

df_graph = pd.DataFrame({"condicion": ["trans", "alter", "recup"], "he": p_he, "z_he": z_he, "sc": p_sc, "z_sc": z_sc})

In [ ]:
df_graph

In [ ]:
h, pcorregido = fdrcorrection(df_graph["he"].values, alpha=0.05, method="indep", is_sorted=False)
h, pcorregido

In [ ]:
h, pcorregido = fdrcorrection(df_graph["sc"].values, alpha=0.05, method="indep", is_sorted=False)
h, pcorregido

In [ ]:
he_mean = np.mean(np.nanmean(he_matrix, axis=2), axis=0)
he_stderr = np.std(np.nanmean(he_matrix, axis=2), axis=0)/np.sqrt(N_subs)
he_per_subject = np.nanmean(he_matrix, axis=2)

plot_prob_points(
    he_per_subject[:,0], 
    he_per_subject[:,1], 
    he_per_subject[:,2], 
    he_per_subject[:,3], 
    facecolors="b", 
    alpha=0.25,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    #zero_line=True,
    #edge_color="black",
    #yticks=[-0.02, 0.000, 0.02, 0.04, 0.06, 0.08, 0.10],
    #ytickslabels=["-0.02", "0.00", "0.02", "0.04", "0.06", "0.08", "0.10"]
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/entropia/figuras/he.png")


In [ ]:
sc_mean = np.mean(np.nanmean(sc_matrix, axis=2), axis=0)
sc_stderr = np.std(np.nanmean(sc_matrix, axis=2), axis=0)/np.sqrt(N_subs)
sc_per_subject = np.nanmean(sc_matrix, axis=2)

plot_prob_points(
    sc_per_subject[:,0], 
    sc_per_subject[:,1], 
    sc_per_subject[:,2], 
    sc_per_subject[:,3], 
    facecolors="b", 
    alpha=0.25,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    #zero_line=True,
    #edge_color="black",
    #yticks=[-0.02, 0.000, 0.02, 0.04, 0.06, 0.08, 0.10],
    #ytickslabels=["-0.02", "0.00", "0.02", "0.04", "0.06", "0.08", "0.10"]
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/entropia/figuras/sc.png")


In [ ]:
he_per_subject = np.nanmean(he_matrix_controles, axis=2)

a = he_per_subject
a

plot_prob_points(
    he_per_subject[[0,1,2,3,4,5,7,8,9],0], 
    he_per_subject[:,1], 
    he_per_subject[[0,1,2,3,5,7,8,9],2], 
    he_per_subject[:,3], 
    facecolors="b", 
    alpha=0.25,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    #zero_line=True,
    #edge_color="black",
    #yticks=[-0.02, 0.000, 0.02, 0.04, 0.06, 0.08, 0.10],
    #ytickslabels=["-0.02", "0.00", "0.02", "0.04", "0.06", "0.08", "0.10"]
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/entropia/figuras/he_controles.png")


In [ ]:
sc_mean = np.mean(np.nanmean(sc_matrix_controles, axis=2), axis=0)
sc_stderr = np.std(np.nanmean(sc_matrix_controles, axis=2), axis=0)/np.sqrt(N_subs)
sc_per_subject = np.nanmean(sc_matrix_controles, axis=2)

plot_prob_points(
    sc_per_subject[:,0], 
    sc_per_subject[:,1], 
    sc_per_subject[:,2], 
    sc_per_subject[:,3], 
    facecolors="b", 
    alpha=0.25,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    #zero_line=True,
    #edge_color="black",
    #yticks=[-0.02, 0.000, 0.02, 0.04, 0.06, 0.08, 0.10],
    #ytickslabels=["-0.02", "0.00", "0.02", "0.04", "0.06", "0.08", "0.10"]
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/entropia/figuras/sc_controles.png")


In [ ]:
he_per_subject = np.nanmean(he_matrix_controles, axis=2)

plt.bar([1,2,3,4], np.nanmean(he_per_subject, axis=0))
plt.ylim([0.79, 0.81])

In [ ]:
he_per_subject = np.nanmean(he_matrix, axis=2)

plt.bar([1,2,3,4], np.nanmean(he_per_subject, axis=0))
plt.ylim([0.89, 0.91])